# Read results datasets

In this notebook all the results obtained will be reported, with particular reference to the best configuration for each model.

## Imports

In [1]:
import itertools
import os.path
from typing import List, Dict

import numpy as np
import pandas as pd

from src.models.config import param_layers, param_grid_mlp, param_layers_batch, param_grid_mlp_batch
from src.utils.const import MODEL_RESULTS_CSV, NETWORK_RESULT_CSV
from typing import Tuple

### Useful path to data

In [2]:
RESULTS_FOLDER = os.path.join('..', MODEL_RESULTS_CSV)
MLP_RESULTS_FOLDER = os.path.join('..', NETWORK_RESULT_CSV)

## Read output csv

In [3]:
mlp_all = pd.read_csv(os.path.join(MLP_RESULTS_FOLDER, 'out_mlp_all.csv'))
mlp_batch = pd.read_csv(os.path.join(MLP_RESULTS_FOLDER, 'out_mlp_batch.csv'))
svm_res = pd.read_csv(os.path.join(RESULTS_FOLDER, 'out_svm.csv'))
naive_res = pd.read_csv(os.path.join(RESULTS_FOLDER, 'best_out_naive_bayes.csv'))
tree_res = pd.read_csv(os.path.join(RESULTS_FOLDER, 'best_out_tree_based.csv'))

## Find best configuration of MLP

### Utility function to explore results DataFrame

We need to find configuration with the best f1-score for each fold.

In [4]:
def find_max_f1_cfg(df: pd.DataFrame) -> List:
    cfg = []
    for fold in df['fold'].unique():
        idx = df[df['fold'] == fold]['f1_test'].idxmax()
        cfg.append(df.iloc[idx]['cfg'])
    cfgs = np.unique(np.array(cfg))
    return cfgs

In [5]:
best_cfg = find_max_f1_cfg(mlp_all)
print(f'Best configuration ID: {best_cfg}')

Best configuration ID: [ 3. 17. 18. 35.]


Having performed cross validation with several test sets, it is possible to obtain the mean value of the specified metric and its confidence interval, with 90% accuracy.

In [6]:
def mu_confidence_interval(data: np.ndarray) -> {}:
    t = 1.64
    mu = np.mean(data)
    standard_deviation = np.std(data)
    M = data.shape[0]
    t_student = t * standard_deviation / np.sqrt(M)
    first_interval = mu - t_student
    second_interval = mu + t_student
    return {
        'mu': mu,
        't_student': t_student,
        'first_interval': first_interval,
        'second_interval': second_interval
    }

In order to find the best configuration between the indexes that we found previously, this function calculate the mean on each configuration between the folds, and select the one which has the higher mean.

In [7]:
def find_best_conf(lst_conf, df: pd.DataFrame) -> dict:
    conf = []
    for idx, cfg in enumerate(lst_conf):
        conf.append(
            {
                'f1': mu_confidence_interval(df[df['cfg'] == cfg]['f1_test']),
                'loss': mu_confidence_interval(df[df['cfg'] == cfg]['f1_test']),
                'acc': mu_confidence_interval(df[df['cfg'] == cfg]['f1_test'])
            }
        )
        conf[idx]['conf'] = cfg
    max = conf[0]

    for elm in conf:
        if max['f1']['mu'] < elm['f1']['mu']:
            max = elm
    return max

Since in the output files containing the results of the neural network, the configuration is stored via the index, it is necessary to recalculate all configurations and select only the one of interest, specifying the index.

In [8]:
def get_best_configuration_mlp(cfg: int, p_layer, p_grid_mlp) -> Tuple:
    hyper_parameters_model_all = itertools.product(
        p_layer['input_act'],
        p_layer['hidden_act'],
        p_layer['hidden_size'],
        p_layer['num_hidden_layers'],
        p_layer['dropout'],
        p_layer['batch_norm'],
        p_layer['output_fn'],
        p_grid_mlp['num_epochs'],
        p_grid_mlp['starting_lr'],
        p_grid_mlp['batch_size'],
        p_grid_mlp['optim'],
        p_grid_mlp['momentum'],
        p_grid_mlp['weight_decay'],
    )
    return list(hyper_parameters_model_all)[cfg]

Utility function to print the calculated statistics and the relative configuration.

In [9]:
def print_statistics_model(dictionary: Dict, model: str):
    print(
        f"Best configuration {model} mean metrics:\n"
        f"f1_score: {dictionary['f1']['mu']} ±{dictionary['f1']['t_student']}\n"
        f"loss: {dictionary['loss']['mu']} ±{dictionary['loss']['t_student']}\n"
        f"acc: {dictionary['acc']['mu']} ±{dictionary['acc']['t_student']}\n\n"
        f"Best hyperparams configuration:"
    )
    if model == "MLP":
        best_cfg_mlp_all = get_best_configuration_mlp(int(dictionary['conf']), param_layers, param_grid_mlp)
        for idx, key in enumerate(param_layers.keys()):
            print(f"{key}: {best_cfg_mlp_all[idx]}")
        for idx, key in enumerate(param_grid_mlp.keys(), 7):
            print(f"{key}: {best_cfg_mlp_all[idx]}")
    else:
        print(f"{dictionary['conf']}")

### Results best cfg mlp

In [10]:
res_mlp = find_best_conf(best_cfg, mlp_all)
print_statistics_model(res_mlp, "MLP")

Best configuration MLP mean metrics:
f1_score: 0.8610175579840659 ±0.007531035084099725
loss: 0.8610175579840659 ±0.007531035084099725
acc: 0.8610175579840659 ±0.007531035084099725

Best hyperparams configuration:
input_act: LeakyReLU(negative_slope=0.01)
hidden_act: LeakyReLU(negative_slope=0.01)
hidden_size: 512
num_hidden_layers: 3
dropout: 0.2
batch_norm: True
output_fn: None
num_epochs: 200
starting_lr: 0.001
batch_size: 128
optim: <class 'torch.optim.adam.Adam'>
momentum: 0.9
weight_decay: 1e-05


#### mlp with different batch_size

In [11]:
mlp_batch

,Unnamed: 0,cfg,fold,loss_test,acc_test,f1_test,mean_loss,std_loss,mean_acc_val,std_acc_val,mean_acc_train,std_acc_train,mean_f1_train,std_f1_train,mean_f1_val,std_f1_val
0,0,0,1,18.940403,13.612167,0.148006,0.289398,0.013900,75.375842,0.475101,88.254775,0.562407,0.881785,0.005654,0.754430,0.003962
1,1,1,1,13.517536,22.813688,0.216087,0.296447,0.022876,75.671540,1.257495,87.909500,0.968884,0.878310,0.009760,0.757455,0.012299
2,2,2,1,18.105448,12.433460,0.142572,0.301010,0.020293,75.823471,1.143876,87.699934,0.865473,0.876271,0.008692,0.758931,0.011460
3,3,3,1,0.751302,67.984791,0.684068,0.410880,0.191540,72.575773,5.793029,83.027949,8.142960,0.828278,0.083659,0.727154,0.056903
4,4,4,1,1.168189,49.581749,0.505397,0.611775,0.437051,66.671148,12.934831,74.474675,18.603783,0.739389,0.193005,0.669061,0.127240
5,5,5,1,2.254341,18.631179,0.154145,0.886728,0.732925,57.278686,24.252981,65.782976,25.818213,0.640302,0.283215,0.570275,0.250451
6,6,6,1,12.516399,19.201521,0.186588,0.806154,0.706682,59.659431,23.200582,68.741819,24.977831,0.672284,0.273661,0.594345,0.239270
7,7,7,1,9.610622,20.152091,0.193252,0.750074,0.677493,61.345407,22.158502,70.762288,23.968632,0.694280,0.262520,0.611319,0.228306
8,8,8,1,8.661695,25.057034,0.219253,0.712597,0.647498,62.370921,21.092658,72.070202,22.899502,0.708747,0.250875,0.621693,0.217251
9,9,9,1,2.770374,25.893536,0.240637,0.730237,0.616622,61.641182,20.130258,71.050573,21.942100,0.699167,0.239762,0.614613,0.207203


In [12]:
df=pd.DataFrame()

In [24]:
for idx in mlp_batch['cfg'].unique():
    config=get_best_configuration_mlp(idx, param_layers_batch, param_grid_mlp_batch)
    batch_norm=config[5]
    batch_size=config[9]
    f1_dict=mu_confidence_interval(mlp_batch[mlp_batch['cfg'] == idx]['f1_test'])
    loss_dict=mu_confidence_interval(mlp_batch[mlp_batch['cfg'] == idx]['loss_test'])
    acc_dict=mu_confidence_interval(mlp_batch[mlp_batch['cfg'] == idx]['acc_test'])

    new_sample=pd.DataFrame({
                'index':[idx],
                'batch_norm':[batch_norm],
                'batch_size':[batch_size],
                'f1_mean': [f1_dict['mu']],
                'f1_confidence': [f1_dict['t_student']],
                'loss_mean': [loss_dict['mu']],
                'loss_confidence': [loss_dict['t_student']],
                'acc_mean': [acc_dict['mu']],
                'acc_confidence':[acc_dict['t_student']]

                })
    df=pd.concat([df,new_sample])
print(f"{df}\n")

   index  batch_norm  batch_size   f1_mean  f1_confidence  loss_mean  \
0      0        True          16  0.285382       0.186657   8.021037   
0      1        True          32  0.310350       0.176673   6.453717   
0      2        True          64  0.400204       0.208004   5.942804   
0      3        True         512  0.446364       0.180163   2.074904   
0      4        True        2048  0.326819       0.122531   1.850459   
0      5        True       16384  0.121477       0.034068   2.276308   
0      6       False          16  0.170163       0.009899   6.918986   
0      7       False          32  0.175928       0.042480   6.227965   
0      8       False          64  0.303729       0.170434   5.000649   
0      9       False         512  0.225762       0.024920   3.205096   
0     10       False        2048  0.364835       0.105409   1.696874   
0     11       False       16384  0.142564       0.014143   2.292822   

   loss_confidence   acc_mean  acc_confidence  
0         4.417

In [14]:
res_mlp = find_best_conf(best_cfg, mlp_all)
print_statistics_model(res_mlp, "MLP")

#### mlp with different batch_size

In [15]:
mlp_batch

## Scikit learn best cfg

In [16]:
def calculate_statistics_sklearn(df: pd.DataFrame, model: str) -> Dict:
    res = {'f1': mu_confidence_interval(df[df['model'] == model]['f1_test']),
           'loss': mu_confidence_interval(df[df['model'] == model]['loss_test']),
           'acc': mu_confidence_interval(df[df['model'] == model]['acc_test']),
           'conf': df[df['model'] == model]['cfg'].unique()}
    return res

NameError: name 'calculate_statistics_sklearn' is not defined

### Tree based

#### Random forest

In [ ]:
res_random_forest = calculate_statistics_sklearn(tree_res, 'random_forest_classifier')
print_statistics_model(res_random_forest, 'random forest classifier')

#### Decision tree

In [ ]:
res_decision_tree = calculate_statistics_sklearn(tree_res, 'decision_tree_classifier')
print_statistics_model(res_decision_tree, 'decision tree classifier')

## Naive bayes

#### Gaussian naive bayes

In [ ]:
res_gaussian_nb = calculate_statistics_sklearn(naive_res, 'gaussian_nb')
print_statistics_model(res_gaussian_nb, 'gaussianNB')

#### QDA

In [16]:
res_qda = calculate_statistics_sklearn(naive_res, 'qda')
print_statistics_model(res_qda, 'QDA')

Best configuration QDA mean metrics:
f1_score: 0.5217235201683785 ±0.006131748988284651
loss: 0.4649740608914607 ±0.0063746149205477646
acc: 0.5350259391085392 ±0.00637461492054775

Best hyperparams configuration:
["{'reg_param': 0.001, 'tol': 0.0001}"]


## SVM

In [20]:
res_svm = calculate_statistics_sklearn(svm_res, 'svc')
print_statistics_model(res_svm, 'SVM')

Best configuration SVM mean metrics:
f1_score: 0.8286206857647119 ±0.003255026164399803
loss: 0.17076133850717423 ±0.0031192905594531074
acc: 0.8292386614928257 ±0.0031192905594531065

Best hyperparams configuration:
["{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}"]
